In [1]:
import os
import shutil
import unittest
from catvae.trainer import MultVAE, BiomDataModule
from catvae.sim import multinomial_bioms
from biom import Table
from biom.util import biom_open
import numpy as np
from pytorch_lightning import Trainer
import argparse
import seaborn as sns

In [2]:
import torch
torch.__version__

'1.8.0'

# Simulate data

In [3]:
np.random.seed(0)
k = 10
sims = multinomial_bioms(k=k, D=50, N=1000, M=50000)
Y = sims['Y']
parts = Y.shape[0] // 10
samp_ids = list(map(str, range(Y.shape[0])))
obs_ids = list(map(str, range(Y.shape[1])))
train = Table(Y[:parts * 8].T, obs_ids, samp_ids[:parts * 8])
test = Table(Y[parts * 8 : parts * 9].T,
             obs_ids, samp_ids[parts * 8 : parts * 9])
valid = Table(Y[parts * 9:].T, obs_ids, samp_ids[parts * 9:])
tree = sims
with biom_open('train.biom', 'w') as f:
    train.to_hdf5(f, 'train')
with biom_open('test.biom', 'w') as f:
    test.to_hdf5(f, 'test')
with biom_open('valid.biom', 'w') as f:
    valid.to_hdf5(f, 'valid')

In [4]:
sims['tree']

<TreeNode, name: y0, internal node count: 48, tips count: 50>

In [5]:
sims['tree'].write('basis.nwk')

'basis.nwk'

# Train model

In [8]:
from pytorch_lightning.profiler import AdvancedProfiler

output_dir = 'output'
args = [
    '--basis', 'basis.nwk',
    '--output-directory', output_dir,
    '--epochs', '50',
    '--num-workers', '10',
    '--scheduler', 'cosine',
    '--learning-rate', '1e-1',
    '--n-latent', str(k),
    '--gpus', '1'
]

dm = BiomDataModule('train.biom', 'test.biom', 'valid.biom',
                    batch_size=50)


parser = argparse.ArgumentParser(add_help=False)
parser = MultVAE.add_model_specific_args(parser)
parser.add_argument('--num-workers', type=int)
parser.add_argument('--gpus', type=int)
args = parser.parse_args(args)
model = MultVAE(args)
model.set_eigs(sims['eigvectors'], sims['eigs'])
profiler = AdvancedProfiler()

trainer = Trainer(
    max_epochs=args.epochs,
    gpus=args.gpus,
    check_val_every_n_epoch=10,
    #profiler=profiler,
    fast_dev_run=False,
    # auto_scale_batch_size='power'
)
trainer.fit(model)

TypeError: ABCMeta object argument after ** must be a mapping, not Namespace

In [ ]:
%load_ext tensorboard
%tensorboard --logdir lightning_logs

In [ ]:
trainer.model

# Evaluate the model

In [ ]:
!ls lightning_logs/version_9/checkpoints

In [ ]:
#W = model.model.get_loadings()
from scipy.stats import pearsonr
import matplotlib.pyplot as plt
from scipy.spatial.distance import pdist

W = model.model.decoder.weight.detach().cpu().numpy()

d_estW = pdist(W)
simW = sims['W'] / np.sqrt(sims['eigs'])
dW = pdist(simW)

plt.scatter(dW, d_estW, s=5)
#plt.plot(np.linspace(0, 4), np.linspace(0, 4), 'r')
plt.xlabel('Predicted correlations')
plt.ylabel('Actual correlations')

print(pearsonr(dW, d_estW))

In [ ]:
model